In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%pip install biopython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 17.7 MB/s eta 0:00:00


In [3]:
!wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
!git clone https://github.com/dauparas/ProteinMPNN
%cd ./ProteinMPNN
!conda create --name mpnn -y

--2024-06-24 01:32:20--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.32.241, 104.16.191.158, 2606:4700::6810:bf9e, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.32.241|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 143808873 (137M) [application/octet-stream]
Saving to: ‘Miniconda3-latest-Linux-x86_64.sh’

Miniconda3-latest-L 100%[===================>] 137.15M   106MB/s    in 1.3s    

2024-06-24 01:32:22 (106 MB/s) - ‘Miniconda3-latest-Linux-x86_64.sh’ saved [143808873/143808873]

PREFIX=/usr/local
Unpacking payload ...

Installing base environment...

Preparing transaction: ...working... done
Executing transaction: ...working... done
installation finished.
    You currently have a PYTHONPATH environment variable set. This may cause
    unexpected behavior when running the Python interpreter in Miniconda3.
    For best results, please verify that your PYTHONPA

In [4]:
%%bash
source activate mpnn
conda install pytorch torchvision torchaudio cudatoolkit=11.3 -c pytorch -y

Channels:
 - pytorch
 - defaults
Platform: linux-64
Solving environment: ...working... done

## Package Plan ##

  environment location: /usr/local/envs/mpnn

  added / updated specs:
    - cudatoolkit=11.3
    - pytorch
    - torchaudio
    - torchvision


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    blas-1.0                   |              mkl           6 KB
    certifi-2024.6.2           |  py312h06a4308_0         161 KB
    cudatoolkit-11.3.1         |       h2bc3f7f_2       549.3 MB
    ffmpeg-4.3                 |       hf484d3e_0         9.9 MB  pytorch
    filelock-3.13.1            |  py312h06a4308_0          24 KB
    freetype-2.12.1            |       h4a9f257_0         626 KB
    gmp-6.2.1                  |       h295c915_3         544 KB
    gnutls-3.6.15              |       he1e5248_0         1.0 MB
    intel-openmp-2023.1.0      |   hdb19cb5_46306        17.2 MB
    

In [5]:
import os
import shutil
import glob
import json
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import uuid
from datetime import datetime
import re
import torch
from time import time

meta_data_filepath = "/content/drive/MyDrive/Generative_Models/IL10_scaffolding/metadata_mpnn_fixed.csv"

if os.path.exists(meta_data_filepath):
  all_metadata_df = pd.read_csv(meta_data_filepath)
  print("Existing generation metadata read in.")
else:
  all_metadata_df = pd.DataFrame()
  #all_metadata_df.to_csv(meta_data_filepath, index=False)
  print("Created mpnn metadata dataframe")

Existing generation metadata read in.


In [6]:
import os
root_dir = "/content/drive/MyDrive/Generative_Models/IL10_scaffolding/"
paths = []
for dirpath, dirnames, filenames in os.walk(root_dir):
  for filename in filenames:
      if "generation_metadata" in filename:
          paths.append(os.path.join(dirpath, filename))

import pandas as pd
all_dfs = []
for file_path in paths:
  df = pd.read_csv(file_path)
  df["dir_path"] = "/".join(file_path.split("/")[:-1])
  all_dfs.append(df)
gen_meta = pd.concat(all_dfs, ignore_index=True)

In [7]:
gen_meta = gen_meta[(gen_meta['entity_id'].notnull()) & (gen_meta['task'].str.contains('backbone'))] #& (gen_meta['model'].isin(['framediff']))]
gen_meta['input_file_path'] = (gen_meta.dir_path + '/' + gen_meta.output_file_name)
gen_meta = gen_meta.rename(columns={"output_file_name":"input_file_name"})
gen_meta = gen_meta[['model','input_file_path','input_file_name', "entity_id"]]


In [8]:
gen_meta

,model,input_file_path,input_file_name,entity_id
0,RFdiffusion_150it,/content/drive/MyDrive/Generative_Models/IL10_...,rfdiffusion_il10_mono_06cc55b4-c549-45f5-b5d0-...,06cc55b4-c549-45f5-b5d0-38f999f290cb
1,RFdiffusion_150it,/content/drive/MyDrive/Generative_Models/IL10_...,rfdiffusion_il10_mono_d858977c-980f-41f6-abdc-...,d858977c-980f-41f6-abdc-10bcb2331680
2,RFdiffusion_150it,/content/drive/MyDrive/Generative_Models/IL10_...,rfdiffusion_il10_mono_96031dd0-9b58-425e-b0db-...,96031dd0-9b58-425e-b0db-a7bd0347cf87
3,RFdiffusion_150it,/content/drive/MyDrive/Generative_Models/IL10_...,rfdiffusion_il10_mono_c44462fd-e20b-4f78-b147-...,c44462fd-e20b-4f78-b147-af279a30af71
4,RFdiffusion_150it,/content/drive/MyDrive/Generative_Models/IL10_...,rfdiffusion_il10_mono_9fb6f4cf-c2cd-4144-9ca4-...,9fb6f4cf-c2cd-4144-9ca4-4b7aca9fa989
...,...,...,...,...
407,RFdiffusion_150it,/content/drive/MyDrive/Generative_Models/IL10_...,rfdiffusion_tev_mono_80b873d2-5b71-4a19-bef9-6...,80b873d2-5b71-4a19-bef9-6cf8a59bd513
408,RFdiffusion_150it,/content/drive/MyDrive/Generative_Models/IL10_...,rfdiffusion_tev_mono_4f9c5216-de0b-4cec-abb0-f...,4f9c5216-de0b-4cec-abb0-fa4158a28c6d
409,RFdiffusion_150it,/content/drive/MyDrive/Generative_Models/IL10_...,rfdiffusion_tev_mono_c25c7794-91e3-4741-8ad6-b...,c25c7794-91e3-4741-8ad6-b5430a0d5021
410,RFdiffusion_150it,/content/drive/MyDrive/Generative_Models/IL10_...,rfdiffusion_tev_mono_df133588-3ef7-4cf2-b2dc-3...,df133588-3ef7-4cf2-b2dc-3fc749fe84bd


In [9]:
meta_data_filepath = "/content/drive/MyDrive/Generative_Models/IL10_scaffolding/metadata_mpnn_solo.csv"

if os.path.exists(meta_data_filepath):
  all_metadata_df = pd.read_csv(meta_data_filepath)
  print("Existing generation metadata read in.")
else:
  all_metadata_df = pd.DataFrame()
  #all_metadata_df.to_csv(meta_data_filepath, index=False)
  print("Created mpnn metadata dataframe")

Created mpnn metadata dataframe


In [16]:
from Bio import SeqIO
import time
num_designs = 100

mpnn_alphabet = 'ACDEFGHIKLMNPQRSTVWYX'
mpnn_alphabet_dict = {'A': 0,'C': 1,'D': 2,'E': 3,'F': 4,'G': 5,'H': 6,'I': 7,'K': 8,'L': 9,'M': 10,'N': 11,'P': 12,'Q': 13,'R': 14,'S': 15,'T': 16,'V': 17,'W': 18,'Y': 19,'X': 20}
chain_length = 184
#chain_length = 237

#pdb_file= "/content/drive/MyDrive/Generative_Models/IL10_scaffolding/tev_monomer.pdb"
pdb_file= "/content/drive/MyDrive/Generative_Models/IL10_scaffolding/IL10_Mutant_model1.pdb"
output_dir = "/content/drive/MyDrive/Generative_Models/IL10_scaffolding/MPNN_solo_redesigns"
output_fasta = output_dir + "/seqs/il10.fa"
os.makedirs(output_dir, exist_ok=True)
#make the json which biases the motif positions completely
bias_per_residue = np.zeros([chain_length, 21])
residues = list(range(24,50)) + list(range(90,125))
#"24/A25-50/40/A91-125/59/0 C1-184"
motif_res = "QSENSCTHFPGYLPNMLRDLRDAFSRLEEVMPQAENQDPDIKAHVISLGENLNTLRLRLRR"
#residues = list(range(27,33)) + list(range(46,51)) + list(range(139,152)) + list(range(167,179)) + list(range(211,221))
#"27/A28-33/13/A47-51/88/A140-152/15/A168-179/32/A212-221/16"
#motif_res = "GHTTSLHLFRRFWKHWIQTKDGQCHSASNFTNTNNYWGGHKVFMVK"
motif_res = [mpnn_alphabet_dict[x] for x in motif_res]
for j, res in enumerate(residues):
  for aa in mpnn_alphabet:
    if mpnn_alphabet_dict[aa] == motif_res[j]:
      bias_per_residue[res,mpnn_alphabet_dict[aa]] = 100
    else:
      bias_per_residue[res,mpnn_alphabet_dict[aa]] = -100
bias_by_res_dict = {}
bias_by_res_dict["A"] = bias_per_residue.tolist()
motif_bias_dict = {}
motif_bias_dict[pdb_file.split("/")[-1].split(".")[0]] = bias_by_res_dict
with open('/content/ProteinMPNN/motif_bias.jsonl', 'w') as f:
  json.dump(motif_bias_dict, f)

meta_data_entry = pd.Series()
meta_data_entry["gen_model"] = "NA"
meta_data_entry["model"] = "ProteinMPNN"
meta_data_entry["input_file_path"] = "NA"
meta_data_entry["task"] = "Sequence Redesign (fixed motif)"
meta_data_entry["Timestamp"] = str(datetime.now())
meta_data_entry['gpu'] = 'T4 GPU'
meta_data_entry['output_file_path'] = output_fasta
meta_data_entry['num_designs'] = num_designs

start_time = time.time()
if "comp" in pdb_file:
  mpnn_command = f"""
    source activate mpnn
    python ./protein_mpnn_run.py --pdb_path {pdb_file} --pdb_path_chains A --out_folder {output_dir} --num_seq_per_target {num_designs} --bias_by_res_jsonl '/content/ProteinMPNN/motif_bias.jsonl' --sampling_temp "0.1" --seed 0 --batch_size 1 --ca_only
    """
else:
  mpnn_command = f"""
    source activate mpnn
    python ./protein_mpnn_run.py --pdb_path {pdb_file} --out_folder {output_dir} --num_seq_per_target {num_designs} --bias_by_res_jsonl '/content/ProteinMPNN/motif_bias.jsonl'  --sampling_temp "0.1" --seed 0 --batch_size 1 --ca_only
    """
!{mpnn_command}
end_time = time.time()
meta_data_entry['wall_time_task'] = str(end_time-start_time) + " Seconds"

try:
  records = list(SeqIO.parse(output_fasta, "fasta"))
  for pair in [pair.split('=') for pair in records[0].description.split(', ')[1:]]:
        meta_data_entry[pair[0]] = pair[1]
  for i, record in enumerate(records):
    #record.id = row['model'] + "_il10_fixed_" + row["entity_id"]+"_design"+str(i)
    record.id = "mpnn_solo_il10_fixed_" + str(uuid.uuid4())
    record.name = "mpnn_solo_il10_fixed_" + str(uuid.uuid4())

  SeqIO.write(records, output_fasta, "fasta")
  all_metadata_df = pd.concat([all_metadata_df,pd.DataFrame(meta_data_entry).T], ignore_index=True)
  all_metadata_df.to_csv(meta_data_filepath, index=False)
except FileNotFoundError:
  print(f"The file {output_fasta} does not exist.")






Using CA-ProteinMPNN!
----------------------------------------
chain_id_jsonl is NOT loaded
----------------------------------------
fixed_positions_jsonl is NOT loaded
----------------------------------------
pssm_jsonl is NOT loaded
----------------------------------------
omit_AA_jsonl is NOT loaded
----------------------------------------
bias_AA_jsonl is NOT loaded
----------------------------------------
tied_positions_jsonl is NOT loaded
bias by residue dictionary is loaded
----------------------------------------
----------------------------------------
Number of edges: 48
Training noise level: 0.2A
/content/ProteinMPNN/protein_mpnn_utils.py:799: UserWarning: Using torch.cross without specifying the dim arg is deprecated.
Please either pass the dim explicitly or simply use torch.linalg.cross.
The default value of dim will change to agree with that of linalg.cross in a future release. (Triggered internally at /opt/conda/conda-bld/pytorch_1716905991019/work/aten/src/ATen/native/C

In [17]:
meta_data_entry

gen_model                                                          NA
model                                                     ProteinMPNN
input_file_path                                                    NA
task                                  Sequence Redesign (fixed motif)
Timestamp                                  2024-06-24 02:03:04.429074
gpu                                                            T4 GPU
output_file_path    /content/drive/MyDrive/Generative_Models/IL10_...
num_designs                                                       100
wall_time_task                             145.08001136779785 Seconds
dtype: object

In [18]:
all_metadata_df = pd.concat([all_metadata_df,pd.DataFrame(meta_data_entry).T], ignore_index=True)

In [19]:
all_metadata_df

,gen_model,model,input_file_path,task,Timestamp,gpu,output_file_path,num_designs,wall_time_task
0,NA,ProteinMPNN,NA,Sequence Redesign (fixed motif),2024-06-24 01:44:48.418461,T4 GPU,/content/drive/MyDrive/Generative_Models/IL10_...,100,172.54332041740417 Seconds
1,NA,ProteinMPNN,NA,Sequence Redesign (fixed motif),2024-06-24 02:03:04.429074,T4 GPU,/content/drive/MyDrive/Generative_Models/IL10_...,100,145.08001136779785 Seconds


In [20]:
all_metadata_df.to_csv(meta_data_filepath, index=False)

In [19]:
from Bio import SeqIO
import time
num_designs = 10

mpnn_alphabet = 'ACDEFGHIKLMNPQRSTVWYX'
mpnn_alphabet_dict = {'A': 0,'C': 1,'D': 2,'E': 3,'F': 4,'G': 5,'H': 6,'I': 7,'K': 8,'L': 9,'M': 10,'N': 11,'P': 12,'Q': 13,'R': 14,'S': 15,'T': 16,'V': 17,'W': 18,'Y': 19,'X': 20}
#chain_length = 184
chain_length = 237

for index, row in gen_meta.iterrows():
  pdb_file= row["input_file_path"]
  output_dir = "/content/drive/MyDrive/Generative_Models/IL10_scaffolding/rfdiffusion_il10/MPNN_redesigns_fixed"
  output_fasta = output_dir + "/seqs/" +row["input_file_name"].split(".")[0]+".fa"
  if 'tev' not in output_fasta: continue
  print(pdb_file)
  if (not all_metadata_df.empty) and (any(all_metadata_df['output_file_path'].str.contains(output_fasta))): continue #file has already been redesigned
  os.makedirs(output_dir, exist_ok=True)

  #make the json which biases the motif positions completely
  bias_per_residue = np.zeros([chain_length, 21])
  #residues = list(range(24,50)) + list(range(90,125))
  #"24/A25-50/40/A91-125/59/0 C1-184"
  #motif_res = "QSENSCTHFPGYLPNMLRDLRDAFSRLEEVMPQAENQDPDIKAHVISLGENLNTLRLRLRR"

  residues = list(range(27,33)) + list(range(46,51)) + list(range(139,152)) + list(range(167,179)) + list(range(211,221))
  #"27/A28-33/13/A47-51/88/A140-152/15/A168-179/32/A212-221/16"
  motif_res = "GHTTSLHLFRRFWKHWIQTKDGQCHSASNFTNTNNYWGGHKVFMVK"

  motif_res = [mpnn_alphabet_dict[x] for x in motif_res]
  for j, res in enumerate(residues):
    for aa in mpnn_alphabet:
      if mpnn_alphabet_dict[aa] == motif_res[j]:
        bias_per_residue[res,mpnn_alphabet_dict[aa]] = 100
      else:
        bias_per_residue[res,mpnn_alphabet_dict[aa]] = -100
  bias_by_res_dict = {}
  bias_by_res_dict["A"] = bias_per_residue.tolist()
  motif_bias_dict = {}
  motif_bias_dict[pdb_file.split("/")[-1].split(".")[0]] = bias_by_res_dict
  with open('/content/ProteinMPNN/motif_bias.jsonl', 'w') as f:
    json.dump(motif_bias_dict, f)

  meta_data_entry = row.copy()[["model","input_file_path","entity_id"]]
  meta_data_entry["gen_model"] = row["model"]
  meta_data_entry["model"] = "ProteinMPNN"
  meta_data_entry["task"] = "Sequence Redesign (fixed motif)"
  meta_data_entry["Timestamp"] = str(datetime.now())
  meta_data_entry['gpu'] = 'T4 GPU'
  meta_data_entry['output_file_path'] = output_fasta
  meta_data_entry['num_designs'] = num_designs

  start_time = time.time()
  if "comp" in pdb_file:
    mpnn_command = f"""
      source activate mpnn
      python ./protein_mpnn_run.py --pdb_path {pdb_file} --pdb_path_chains A --out_folder {output_dir} --num_seq_per_target {num_designs} --bias_by_res_jsonl '/content/ProteinMPNN/motif_bias.jsonl' --sampling_temp "0.1" --seed 0 --batch_size 1 --ca_only
      """
  else:
    mpnn_command = f"""
      source activate mpnn
      python ./protein_mpnn_run.py --pdb_path {pdb_file} --out_folder {output_dir} --num_seq_per_target {num_designs} --bias_by_res_jsonl '/content/ProteinMPNN/motif_bias.jsonl'  --sampling_temp "0.1" --seed 0 --batch_size 1 --ca_only
      """
  !{mpnn_command}
  end_time = time.time()
  meta_data_entry['wall_time_task'] = str(end_time-start_time) + " Seconds"

  try:
    records = list(SeqIO.parse(output_fasta, "fasta"))
    for pair in [pair.split('=') for pair in records[0].description.split(', ')[1:]]:
          meta_data_entry[pair[0]] = pair[1]
    for i, record in enumerate(records):
      #record.id = row['model'] + "_il10_fixed_" + row["entity_id"]+"_design"+str(i)
      record.id = row['model'] + "_tev_fixed_" + row["entity_id"]+"_design"+str(i)
      record.name = row['input_file_name']

    SeqIO.write(records, output_fasta, "fasta")
    all_metadata_df = pd.concat([all_metadata_df,pd.DataFrame(meta_data_entry).T], ignore_index=True)
    all_metadata_df.to_csv(meta_data_filepath, index=False)
  except FileNotFoundError:
    print(f"The file {output_fasta} does not exist.")






/content/drive/MyDrive/Generative_Models/IL10_scaffolding/rfdiffusion_il10/rfdiffusion_tev_mono_8d72f918-8456-442f-80c4-d4bb75e2f83d.pdb
Using CA-ProteinMPNN!
----------------------------------------
chain_id_jsonl is NOT loaded
----------------------------------------
fixed_positions_jsonl is NOT loaded
----------------------------------------
pssm_jsonl is NOT loaded
----------------------------------------
omit_AA_jsonl is NOT loaded
----------------------------------------
bias_AA_jsonl is NOT loaded
----------------------------------------
tied_positions_jsonl is NOT loaded
bias by residue dictionary is loaded
----------------------------------------
----------------------------------------
Number of edges: 48
Training noise level: 0.2A
/content/ProteinMPNN/protein_mpnn_utils.py:799: UserWarning: Using torch.cross without specifying the dim arg is deprecated.
Please either pass the dim explicitly or simply use torch.linalg.cross.
The default value of dim will change to agree with 

In [20]:
import os
import shutil
import glob
import json
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import uuid
from datetime import datetime
import re
import torch
from time import time

meta_data_filepath = "/content/drive/MyDrive/Generative_Models/IL10_scaffolding/metadata_mpnn.csv"

if os.path.exists(meta_data_filepath):
  all_metadata_df = pd.read_csv(meta_data_filepath)
  print("Existing generation metadata read in.")
else:
  all_metadata_df = pd.DataFrame()
  #all_metadata_df.to_csv(meta_data_filepath, index=False)
  print("Created mpnn metadata dataframe")

Existing generation metadata read in.


In [21]:
from Bio import SeqIO
import time
num_designs = 10

#mpnn_alphabet = 'ACDEFGHIKLMNPQRSTVWYX'
#mpnn_alphabet_dict = {'A': 0,'C': 1,'D': 2,'E': 3,'F': 4,'G': 5,'H': 6,'I': 7,'K': 8,'L': 9,'M': 10,'N': 11,'P': 12,'Q': 13,'R': 14,'S': 15,'T': 16,'V': 17,'W': 18,'Y': 19,'X': 20}
#chain_length = 184

for index, row in gen_meta.iterrows():
  pdb_file= row["input_file_path"]
  output_dir = "/content/drive/MyDrive/Generative_Models/IL10_scaffolding/rfdiffusion_il10/MPNN_redesigns"
  output_fasta = output_dir + "/seqs/" +row["input_file_name"].split(".")[0]+".fa"
  if 'tev' not in output_fasta: continue
  print(pdb_file)
  if (not all_metadata_df.empty) and (any(all_metadata_df['output_file_path'].str.contains(output_fasta))): continue #file has already been redesigned
  os.makedirs(output_dir, exist_ok=True)

  #make the json which biases the motif positions completely
  #bias_per_residue = np.zeros([chain_length, 21])
  #residues = list(range(24,50)) + list(range(90,125))
  #motif_res = "QSENSCTHFPGYLPNMLRDLRDAFSRLEEVMPQAENQDPDIKAHVISLGENLNTLRLRLRR"
  #motif_res = [mpnn_alphabet_dict[x] for x in motif_res]
  #for j, res in enumerate(residues):
  #  for aa in mpnn_alphabet:
  #    if mpnn_alphabet_dict[aa] == motif_res[j]:
  #      bias_per_residue[res,mpnn_alphabet_dict[aa]] = 100
  #    else:
  #      bias_per_residue[res,mpnn_alphabet_dict[aa]] = -100
  #bias_by_res_dict = {}
  #bias_by_res_dict["A"] = bias_per_residue.tolist()
  #motif_bias_dict = {}
  #motif_bias_dict[pdb_file.split("/")[-1].split(".")[0]] = bias_by_res_dict
  #with open('/content/ProteinMPNN/motif_bias.jsonl', 'w') as f:
  #  json.dump(motif_bias_dict, f)

  meta_data_entry = row.copy()[["model","input_file_path","entity_id"]]
  meta_data_entry["gen_model"] = row["model"]
  meta_data_entry["model"] = "ProteinMPNN"
  meta_data_entry["task"] = "Sequence Redesign"
  meta_data_entry["Timestamp"] = str(datetime.now())
  meta_data_entry['gpu'] = 'T4 GPU'
  meta_data_entry['output_file_path'] = output_fasta
  meta_data_entry['num_designs'] = num_designs

  start_time = time.time()
  if "comp" in pdb_file:
    mpnn_command = f"""
      source activate mpnn
      python ./protein_mpnn_run.py --pdb_path {pdb_file} --pdb_path_chains A --out_folder {output_dir} --num_seq_per_target {num_designs} --sampling_temp "0.1" --seed 0 --batch_size 1 --ca_only
      """
  else:
    mpnn_command = f"""
      source activate mpnn
      python ./protein_mpnn_run.py --pdb_path {pdb_file} --out_folder {output_dir} --num_seq_per_target {num_designs} --sampling_temp "0.1" --seed 0 --batch_size 1 --ca_only
      """
  !{mpnn_command}
  end_time = time.time()
  meta_data_entry['wall_time_task'] = str(end_time-start_time) + " Seconds"

  try:
    records = list(SeqIO.parse(output_fasta, "fasta"))
    for pair in [pair.split('=') for pair in records[0].description.split(', ')[1:]]:
          meta_data_entry[pair[0]] = pair[1]
    for i, record in enumerate(records):
      record.id = row['model'] + "_tev_" + row["entity_id"]+"_design"+str(i)
      record.name = row['input_file_name']

    SeqIO.write(records, output_fasta, "fasta")
    all_metadata_df = pd.concat([all_metadata_df,pd.DataFrame(meta_data_entry).T], ignore_index=True)
    all_metadata_df.to_csv(meta_data_filepath, index=False)
  except FileNotFoundError:
    print(f"The file {output_fasta} does not exist.")






Streaming output truncated to the last 5000 lines.
Number of edges: 48
Training noise level: 0.2A
/content/ProteinMPNN/protein_mpnn_utils.py:799: UserWarning: Using torch.cross without specifying the dim arg is deprecated.
Please either pass the dim explicitly or simply use torch.linalg.cross.
The default value of dim will change to agree with that of linalg.cross in a future release. (Triggered internally at /opt/conda/conda-bld/pytorch_1716905991019/work/aten/src/ATen/native/Cross.cpp:62.)
  n_2 = F.normalize(torch.cross(u_2, u_1), dim=-1)
Generating sequences for: rfdiffusion_tev_mono_8984e14a-be34-418d-a843-d26b034821d1
10 sequences of length 237 generated in 17.9334 seconds
/content/drive/MyDrive/Generative_Models/IL10_scaffolding/rfdiffusion_il10/rfdiffusion_tev_mono_5e2aab60-77a2-4626-8a51-5c2b2f5f8320.pdb
Using CA-ProteinMPNN!
----------------------------------------
chain_id_jsonl is NOT loaded
----------------------------------------
fixed_positions_jsonl is NOT loaded
------

In [ ]:
"""
    records = records[1:]
    for record in records:
      record.id = row['model'] + "-" + row['output_file_name']
    with open(merged_fasta_file, "a") as output_handle:
      SeqIO.write(records, output_handle, "fasta")
"""

In [ ]:
# prompt: kill the colab runtime

!kill -9 -1
